<a href="https://colab.research.google.com/github/CristopherCano/Proyecto_Python_Procesamiento_de_datos/blob/main/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Procesamiento de Datos con Python - BEDU - Módulo 3
## Viajes en Taxis y Ubers en la Zona Metropolitana del Valle de México 2016 - 2017
* Eduardo Alán Hernández Villasana
* Cristopher Arvizu Cano
* David Gil Peralta
* José Antonio Aguilar Téllez

# 1.Planteamiento del Problema
En los últimos años, desde la llegada de aplicaciones para el transporte privado como Uber o Didi, los gremios taxistas de la Zona Metropolitana del Valle de México han protestado en contra de estas aplicaciones, ellos argumentan una competencia desleal y exigen regulaciones hacia estos transportes. Esto no es de extrañarse ya que, con el aumento de la popularidad de estas aplicaciones de transporte, los taxistas están perdiendo clientes y no pueden o quieren modernizarse para poder competir con los automóviles de Uber o Didi que son más modernos. 

A raíz de este enfrentamiento, han surgido noticias comparando estos dos modelos de transporte privado, sin embargo, muchas de estas noticias están sesgadas por intereses políticos y económicos. Es por esta razón que decidimos analizar datos recopilados tanto en Taxis y Ubers para hacer una comparación entre los dos modelos y aportar nuestros hallazgos estadísticos a la discusión de este problema.

## Preguntas Clave
Durante el desarrollo de este proyecto, se espera responder las siguientes preguntas.

* ¿Qué tipo de transporte pasa menos tiempo en el tráfico?
* ¿Cuál es la velocidad promedio de cada tipo de transporte?
* ¿Qué día de la semana hay mas demanda para cada tipo de transporte?
* ¿En qué tipo de transporte recibe más dinero en promedio por viaje?
* ¿Desde qué demsarcación territorial o municipio cada tipo de transporte tiene mayor demanda?
* ¿Cuál es el destino mas solicitado para cada tipo de transporte?

In [ ]:
# Librerías a usar
import pandas as pd
import numpy as np
import datetime as dt

# 2.Extracción de datos
Se decidió usar una base de datos que contenga datos tanto de transportes *Uber* y *Taxis* comunes, se encontró una base de datos con estas características en el sitio kaggle.com. Estos datos fueron recolectados por *EC Taximeter* usando su aplicación movil, ellos recolectaron datos de viajes en Ubers y Taxis de la Ciudad de México de junio de 2016 a julio de 2017.
Se descargará la base de datos desde GitHub.

In [ ]:
url = "https://raw.githubusercontent.com/CristopherCano/Proyecto_Python_Procesamiento_de_datos/main/Data/cdmx_transporte_raw.csv"

In [ ]:
data_raw = pd.read_csv(url, index_col=0)
data_raw.head(2)

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,,
1,México DF Taxi de Sitio,2016-09-16 07:14:12,2016-09-18 04:41:40,-99.097369,19.416874,-99.202729,19.430353,N,120449,12373,242
2,México DF Taxi Libre,2016-09-18 06:16:33,2016-09-18 10:11:43,-99.297148,19.322128,-99.289949,19.326538,N,14110,1700,461


### Descripción de las Columnas
* **id**: Un identificador único para cada viaje.
* **vendor_id**: El tipo de transporte en que se realizó el viaje.
* **pickup_datetime**: Fecha y Hora en que el viaje inició.
* **dropoff_datetime**: Fecha y Hora en que el viaje finalizó.
* **pickup_longitude**: Longitud en la que el viaje inició.
* **pickup_latitude**: Latitud en la que el viaje inició.
* **dropoff_longitude**: Longitud en la que el viaje finalizó.
* **dropoff_latitude**: Latitud en la que el viaje finalizó.
* **store_and_fwd_flag**: Indica si la información del viaje se almacenó en una memoria y después se envio al servidor por que no tenía conexión a internet o no. Todas las entradas de datos estan en 'N'.
* **trip_duration**: Duración del viaje en segundos.
* **dist_meters**: Distancia recorrida en el viaje en metros.
* **wait_sec**: Tiempo en segundos en el que el automovil estuvo completamente detenido durante el viaje. (Se usará como medición del tráfico.)

## Exploración básica de datos
 

OBJETIVO 

- Leer nuestro dataset en un `DataFrame` de `pandas`.
- Realizar Análisis Exploratorio de Datos básico para conocer algunas de las características de nuestro conjunto de datos.

DESARROLLO

En esta sección vamos a empezar a analizar nuestro conjunto de datos usando pandas.

Algunas de las preguntas que estamos intentando responder en esta exploración son las siguientes:

1. ¿El conjunto de datos que tengo realmente me sirve para responder algunas de las preguntas que me planteé?
2. ¿Qué tamaño tiene mi conjunto de datos? ¿Serán datos suficientes?
3. ¿Qué columnas tengo y qué información tengo en cada una de esas columnas?
4. Los nombres que tienen mis columnas, ¿son el nombre más apropiado?
4. ¿Qué tipos de datos tengo en cada columna? ¿Parecen ser el tipo correcto de datos? ¿O es un tipo de datos "incorrecto"?
5. Si selecciono algunas filas al azar y las observo, ¿estoy obteniendo los datos que debería? ¿O hay datos que parecen estar "sucios" o "incorrectos"?

1. ¿El conjunto de datos que tengo realmente me sirve para responder algunas de las preguntas que me planteé?

In [ ]:
tamaño = data_raw.shape
print(f'2. ¿Qué tamaño tiene mi conjunto de datos? \n número de filas: {tamaño[0]} \n número de columnas: {tamaño[1]}')

2. ¿Qué tamaño tiene mi conjunto de datos? 
 número de filas: 12694 
 número de columnas: 11


In [ ]:
print(f'3. ¿Qué columnas tengo y qué información tengo en cada una de esas columnas? \n \n En total tenemos{tamaño[1]} columnas \n \n {data_raw.columns}')

3. ¿Qué columnas tengo y qué información tengo en cada una de esas columnas? 
 
 En total tenemos11 columnas 
 
 Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'store_and_fwd_flag', 'trip_duration', 'dist_meters', 'wait_sec'],
      dtype='object')


4. Los nombres que tienen mis columnas, ¿son el nombre más apropiado?

Podemos observar que algunos nombres como vendor_id o dist_meters no son los más adecuados por lo que tendremos que usar la conveción apropiada que es snake-case y ajustar sus nombres

5. ¿Qué tipos de datos tengo en cada columna? ¿Parecen ser el tipo correcto de datos? ¿O es un tipo de datos "incorrecto"?



In [ ]:
data_raw.dtypes

vendor_id              object
pickup_datetime        object
dropoff_datetime       object
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dist_meters             int64
wait_sec                int64
dtype: object

Las fechas aparecen como object por lo que sera necesario transformar esta variable a un formato de fecha mediante padas

6. Si selecciono algunas filas al azar y las observo, ¿estoy obteniendo los datos que debería? ¿O hay datos que parecen estar "sucios" o "incorrectos"?


In [ ]:
data_raw.sample(5)

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,,
9082,México DF Taxi Libre,2017-01-17 10:21:10,2017-01-17 10:46:38,-99.068458,19.330774,-99.085482,19.382532,N,1529,8345,536
2372,México DF Taxi Libre,2017-04-18 10:43:49,2017-04-18 11:19:57,-99.121888,19.408144,-99.219631,19.415787,N,2169,16175,553
5191,México DF Taxi Libre,2017-06-13 03:28:46,2017-06-13 03:40:24,-99.196089,19.461315,-99.181378,19.447197,N,699,3080,308
1231,México DF Taxi Libre,2017-03-14 09:51:25,2017-03-14 10:09:08,-99.251659,19.305254,-99.239157,19.317197,N,1063,3079,457
10770,México DF Radio Taxi,2017-06-27 12:14:05,2017-06-27 12:21:59,-99.164418,19.417229,-99.155408,19.440062,N,474,2960,233


In [ ]:
### Mi función summary
def summary(nombre):
  minimo = data_raw[nombre].min()
  Q1 = data_raw[nombre].quantile(0.25)
  median = data_raw[nombre].median()
  mean = data_raw[nombre].mean()
  Q2 = data_raw[nombre].quantile(0.5)
  Q3 = data_raw[nombre].quantile(0.75)
  maximo = data_raw[nombre].max()
  IQR = Q3 - Q1
  resumen = {'minimo': minimo, 'Q1': Q1, 'mediana':median,'Q2':Q2,'Q3':Q3,'max':maximo,'IQR':IQR, 'media':mean}
  return resumen

#### Trip duration

In [ ]:
print('trip_dutario_seconds_resumen')
trip_dutario_seconds_resumen = summary('trip_duration')
trip_dutario_seconds_resumen

trip_dutario_seconds_resumen


{'IQR': 1884.75,
 'Q1': 484.0,
 'Q2': 1033.5,
 'Q3': 2368.75,
 'max': 16570949,
 'media': 22983.77138805735,
 'mediana': 1033.5,
 'minimo': 2}

In [ ]:
#Conversion a dia trip_duration
print('duración del viaje maximo en min')
trip_dutario_seconds_resumen['max']/(60)

duración del viaje maximo en min


276182.48333333334

#### dist_meters

In [ ]:
print('dist_meters_resumen')
dist_meters_resumen = summary('dist_meters')
dist_meters_resumen

dist_meters_resumen


{'IQR': 6490.0,
 'Q1': 1775.0,
 'Q2': 3948.0,
 'Q3': 8265.0,
 'max': 802537,
 'mediana': 3948.0,
 'minimo': 21}

In [ ]:
# Coversión de dist_meters en km 
print('distacia maxima rrecorrida en km')
dist_meters_resumen['max']/(1000)

distacia maxima rrecorrida en km


802.537

#### wait_sec

In [ ]:
print('wait_sec_resumen')
wait_sec_resumen = summary('wait_sec')
wait_sec_resumen

wait_sec_resumen


{'IQR': 502.0,
 'Q1': 89.0,
 'Q2': 234.0,
 'Q3': 591.0,
 'max': 4429346278,
 'media': 1161269.6217110446,
 'mediana': 234.0,
 'minimo': 0}

In [ ]:
# Conversión wait_sec a dias 
print('tiempo de espera en minutos')
wait_sec_resumen['max']/(60)

tiempo de espera en minutos


73822437.96666667

## Hallazgos preliminares encontraros

Columnas que tienen valores atípicos: 

- trip_duration,
- dist_meters, 
- wait_sec. 

Sus valores máximos son extremadamente altos, por ejemplo:

- El valor máximo de duración del viaje es ~ 276,182 min, esta cantidad equivale a un viaje de 191 dias, lo cuál es poco confiable.
- El valor máximo de distancia del viaje es ~ 802.54 km, esto equivaldría a un viaje de la CDMX a Monterrey.
- El timpo máximo de espera del viaje es 73,822,437 min, lo que no puede ser cierto para ningún viaje


A continuación observamos los siguiente problemas:
* **vendor_id**: Se repite el patrón México DF
* **store_and_fwd_flag**: No es significativa esta variable ya que siempre marca "N" en cada fila
* **trip_duration**: Tenemos duraciones de viaje exageradamente grandes
* **dist_meters**: Tenemos distacias de viaje exageradamente grandes
* **wait_sec**: Tenemos tiempos de espera de viaje exageradamente grandes

#3.Limpieza de datos y agregaciones

OBJETIVO 

- Limpiar nuestro dataset de `NaNs`.
- Reindexar si es necesario
- Renombrar columnas si es necesario
- Experimentar la aplicación de agregaciones para explorar nuestro dataset

DESARROLLO
- Eliminar datos atipicos
- Limpia del dataset de manera que no quede ningún `NaN`.
- Renombrar columnas

In [ ]:
# Aqui va la corrección de nombres de las columnas
nuevos_nombres = {'vendor_id':'tipo_transporte'}
data_raw2 = data_raw.rename(columns=nuevos_nombres)
data_raw2.head()

,tipo_transporte,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,,
1,México DF Taxi de Sitio,2016-09-16 07:14:12,2016-09-18 04:41:40,-99.097369,19.416874,-99.202729,19.430353,N,120449,12373,242
2,México DF Taxi Libre,2016-09-18 06:16:33,2016-09-18 10:11:43,-99.297148,19.322128,-99.289949,19.326538,N,14110,1700,461
3,México DF Taxi Libre,2016-09-18 10:11:50,2016-09-18 10:23:11,-99.289603,19.326263,-99.271874,19.328530,N,681,2848,129
4,México DF Taxi Libre,2016-09-18 10:23:38,2016-09-18 10:30:53,-99.271161,19.328875,-99.279900,19.326256,N,436,1409,106
5,México DF Taxi Libre,2016-09-18 10:44:18,2016-09-18 10:51:40,-99.282761,19.326944,-99.291705,19.322754,N,442,1567,85


### Chequeo NaN's

In [ ]:
data_raw2.isna().sum(axis=0)

tipo_transporte       0
pickup_datetime       0
dropoff_datetime      0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dist_meters           0
wait_sec              0
dtype: int64

Se revisio la prosible existencia de datos nulos o NaN pero no se encontro este problema.

### Eliminación de columnas

`store_and_fwd_flag`: Es una columna que tiene un unico valor y no muestra ser relevante para un futuro uso por lo que se decide droppear esta columna

In [ ]:
data_raw_dropped = data_raw2.drop(['store_and_fwd_flag'], axis=1)

In [ ]:
 data_raw_dropped['tipo_transporte'] = data_raw_dropped['tipo_transporte'].str.replace("México DF ","")

In [ ]:
data_raw_dropped.head(10)

,tipo_transporte,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,dist_meters,wait_sec
id,,,,,,,,,,
1,Taxi de Sitio,2016-09-16 07:14:12,2016-09-18 04:41:40,-99.097369,19.416874,-99.202729,19.430353,120449,12373,242
2,Taxi Libre,2016-09-18 06:16:33,2016-09-18 10:11:43,-99.297148,19.322128,-99.289949,19.326538,14110,1700,461
3,Taxi Libre,2016-09-18 10:11:50,2016-09-18 10:23:11,-99.289603,19.326263,-99.271874,19.328530,681,2848,129
4,Taxi Libre,2016-09-18 10:23:38,2016-09-18 10:30:53,-99.271161,19.328875,-99.279900,19.326256,436,1409,106
5,Taxi Libre,2016-09-18 10:44:18,2016-09-18 10:51:40,-99.282761,19.326944,-99.291705,19.322754,442,1567,85
6,Taxi Libre,2016-09-18 10:58:07,2016-09-18 10:59:46,-99.289712,19.322929,-99.289280,19.325978,100,797,19
7,Taxi Libre,2016-09-18 11:00:00,2016-09-18 11:05:45,-99.285391,19.326247,-99.282549,19.328117,345,676,169
8,Taxi Libre,2016-09-18 11:08:03,2016-09-18 11:17:06,-99.285891,19.330022,-99.289828,19.319170,544,3771,37
9,Radio Taxi,2016-09-18 01:06:08,2016-09-18 01:26:34,-99.182346,19.370885,-99.176534,19.341410,1226,5662,572


### Casting fechas
En la exploración de datos se observó que las columnas `pickup_datetime` y `dropoff_datetime` son strings, como se planea usar las fechas de los viajes se cambiará el tipo de dato de dichas columnas a `datetime`.

In [ ]:
data_raw_dropped['pickup_datetime'] = pd.to_datetime(data_raw_dropped['pickup_datetime'], unit = 'ns')
data_raw_dropped['pickup_datetime'].head()

id
1   2016-09-16 07:14:12
2   2016-09-18 06:16:33
3   2016-09-18 10:11:50
4   2016-09-18 10:23:38
5   2016-09-18 10:44:18
Name: pickup_datetime, dtype: datetime64[ns]

In [ ]:
data_raw_dropped['dropoff_datetime'] = pd.to_datetime(data_raw_dropped['dropoff_datetime'], unit = 'ns')
data_raw_dropped['dropoff_datetime'].head()

id
1   2016-09-18 04:41:40
2   2016-09-18 10:11:43
3   2016-09-18 10:23:11
4   2016-09-18 10:30:53
5   2016-09-18 10:51:40
Name: dropoff_datetime, dtype: datetime64[ns]

In [ ]:
data_raw_dropped.dtypes

tipo_transporte              object
pickup_datetime      datetime64[ns]
dropoff_datetime     datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
trip_duration                 int64
dist_meters                   int64
wait_sec                      int64
dtype: object

### Conversión de unidades
Las columnas `dist_meters` y `trip_duration` estan en metros y segundos respectivamente. Se prevee que se vayan a ocupar en kilómetros y horas para análisis posteriores, por eso se les someterá a una transformación usando el método `apply`. Como la conversión de unidades es una operación matemática simple, se usarán funciones anónimas *`lambda`*

In [ ]:
data_filtered = data_raw_dropped.copy()

Conversión de la duración del viaje de segundos a horas.

In [ ]:
data_filtered["trip_duration"].head(2)

id
1    120449
2     14110
Name: trip_duration, dtype: int64

In [ ]:
# Como es una operación simple se usará una función anónima.
data_filtered["trip_duration"] = data_filtered["trip_duration"].apply(lambda x: x/3600)
data_filtered["trip_duration"].head(2)

id
1    33.458056
2     3.919444
Name: trip_duration, dtype: float64

Conversión de la distancia recorrida en metros a kilómetros

In [ ]:
data_filtered["dist_meters"].head(2)

id
1    12373
2     1700
Name: dist_meters, dtype: int64

In [ ]:
# Como es una operación simple se usará una función anónima.
data_filtered["dist_km"] = data_filtered["dist_meters"].apply(lambda x: x/1000)
data_filtered["dist_km"].head(2)

id
1    12.373
2     1.700
Name: dist_km, dtype: float64

Creación de la columna `speed_km` que contiene la velocidad del viaje.

In [ ]:
data_filtered["speed_km"] = data_filtered["dist_km"]/data_filtered["trip_duration"]
data_filtered[["dist_km","trip_duration","speed_km"]].head()

,dist_km,trip_duration,speed_km
id,,,
1,12.373,33.458056,0.369806
2,1.700,3.919444,0.433735
3,2.848,0.189167,15.055507
4,1.409,0.121111,11.633945
5,1.567,0.122778,12.762896


Como se puede ver, hay viajes que duran más de lo anormal, debido al contexto del dataframe, consideramos que se deben de eliminar los viajes con velocidades mayores a 200 km/h y con distancias menores a 500 metros (.5 km). Después se eliminarán outliers con el método del Rango Inter Cuartílico.

### Eliminación de outlayers
En la exploración básica de los datos, se observó que hay datos irreales en `trip_duration`, `dist_meters` y en la velocidad. Se decidió hacer un filtro para eliminar los *outliers* usando el método del **Rango Inter Cuartílico (IQR)**. Este método se aplicará a las columnas `wait_sec`, `speed_km`, `dist_km` y `trip_duration`.

In [ ]:
data_filtered = data_filtered[(data_filtered["speed_km"] < 200)  & (data_filtered["dist_km"] > 0.5)]

In [ ]:
Q1 = data_filtered['wait_sec'].quantile(0.25)
Q2 = data_filtered['wait_sec'].quantile(0.5)
Q3 = data_filtered['wait_sec'].quantile(0.75)
IQR_trip_duration = Q3 - Q1
print ("Cuartiles\t\tValor")
print("25%\t\t",Q1)
print("50%\t\t",Q2)
print("75%\t\t",Q3)
print("\n\nRango Intercuartílico: ",IQR_trip_duration)

Cuartiles		Valor
25%		 105.0
50%		 259.0
75%		 623.0


Rango Intercuartílico:  518.0


In [ ]:
data_filtered = data_filtered[(data_filtered["wait_sec"] >= Q1 - 1.5*IQR_trip_duration) & (data_filtered["wait_sec"] <= Q3 + 1.5*IQR_trip_duration)]

In [ ]:
data_filtered.shape

(10142, 12)

#### `speed_km`

In [ ]:
Q1 = data_filtered["speed_km"].quantile(0.25)
Q2 = data_filtered["speed_km"].quantile(0.5)
Q3 = data_filtered["speed_km"].quantile(0.75)
IQR_speed = Q3 - Q1
print ("Cuartiles\t\tValor")
print("25%\t\t",Q1)
print("50%\t\t",Q2)
print("75%\t\t",Q3)
print("\n\nRango Intercuartílico: ",IQR_speed)

Cuartiles		Valor
25%		 10.638143978293026
50%		 16.13962025316456
75%		 21.655161064969338


Rango Intercuartílico:  11.017017086676312


In [ ]:
data_filtered = data_filtered[(data_filtered["speed_km"] >= Q1 - 1.5*IQR_speed) & (data_filtered["speed_km"] <= Q3 + 1.5*IQR_speed)]

#### `dist_km`

In [ ]:
Q1 = data_filtered["dist_km"].quantile(0.25)
Q2 = data_filtered["dist_km"].quantile(0.5)
Q3 = data_filtered["dist_km"].quantile(0.75)
IQR_dist = Q3 - Q1
print ("Cuartiles\t\tValor")
print("25%\t\t",Q1)
print("50%\t\t",Q2)
print("75%\t\t",Q3)
print("\n\nRango Intercuartílico: ",IQR_dist)

Cuartiles		Valor
25%		 2.241
50%		 4.125
75%		 7.744


Rango Intercuartílico:  5.503


In [ ]:
data_filtered = data_filtered[(data_filtered["dist_km"] >= Q1 - 1.5*IQR_dist) & (data_filtered["dist_km"] <= Q3 + 1.5*IQR_dist)]

#### `trip_duration`

In [ ]:
Q1 = data_filtered["trip_duration"].quantile(0.25)
Q2 = data_filtered["trip_duration"].quantile(0.5)
Q3 = data_filtered["trip_duration"].quantile(0.75)
IQR_trip_duration = Q3 - Q1
print ("Cuartiles\t\tValor")
print("25%\t\t",Q1)
print("50%\t\t",Q2)
print("75%\t\t",Q3)
print("\n\nRango Intercuartílico: ",IQR_trip_duration)

Cuartiles		Valor
25%		 0.1461111111111111
50%		 0.2598611111111111
75%		 0.4639583333333333


Rango Intercuartílico:  0.31784722222222217


In [ ]:
data_filtered = data_filtered[(data_filtered["trip_duration"] >= Q1 - 1.5*IQR_trip_duration) & (data_filtered["trip_duration"] <= Q3 + 1.5*IQR_trip_duration)]

#### `wait_sec`

In [ ]:
Q1 = data_filtered['wait_sec'].quantile(0.25)
Q2 = data_filtered['wait_sec'].quantile(0.5)
Q3 = data_filtered['wait_sec'].quantile(0.75)
IQR_trip_duration = Q3 - Q1
print ("Cuartiles\t\tValor")
print("25%\t\t",Q1)
print("50%\t\t",Q2)
print("75%\t\t",Q3)
print("\n\nRango Intercuartílico: ",IQR_trip_duration)

Cuartiles		Valor
25%		 91.0
50%		 205.0
75%		 415.0


Rango Intercuartílico:  324.0


In [ ]:
data_filtered = data_filtered[(data_filtered["wait_sec"] >= Q1 - 1.5*IQR_trip_duration) & (data_filtered["wait_sec"] <= Q3 + 1.5*IQR_trip_duration)]

#### Resumen posterior a eliminación de *outliers*
Después de el proceso de eliminación de los *outliers* se obtienen un resumen de los cuartiles y otros parámetros estadísticos para las columnas `wait_sec`, `speed_km`, `dist_km` y `trip_duration`..

In [ ]:
def summary(nombre):
  minimo = data_filtered[nombre].min()
  Q1 = data_filtered[nombre].quantile(0.25)
  median = data_filtered[nombre].median()
  mean = data_filtered[nombre].mean()
  Q2 = data_filtered[nombre].quantile(0.5)
  Q3 = data_filtered[nombre].quantile(0.75)
  maximo = data_filtered[nombre].max()
  IQR = Q3 - Q1
  resumen = {'minimo': round(minimo,3), 'Q1': round(Q1,3), 'mediana':round(median,3),'Q2':round(Q2),'Q3':round(Q3,3),'max':round(maximo,3),'IQR':round(IQR,3), 'meadi':mean}

  return resumen

In [ ]:
print('velocidad del vaije en km')
speed_km = summary('speed_km')
speed_km_max = speed_km['max']
speed_km

velocidad del vaije en km


{'IQR': 8.648,
 'Q1': 12.963,
 'Q2': 17,
 'Q3': 21.61,
 'max': 38.126,
 'meadi': 17.61683396408097,
 'mediana': 17.045,
 'minimo': 0.737}

In [ ]:
print(f'velocidad máxima del viaje: {speed_km_max} km/h')

velocidad máxima del viaje: 38.126 km/h


In [ ]:
print('distancia del viaje en km')
dist_km = summary('dist_km')
dist_km_max = dist_km['max']
dist_km

distancia del viaje en km


{'IQR': 3.991,
 'Q1': 1.97,
 'Q2': 3,
 'Q3': 5.961,
 'max': 15.998,
 'meadi': 4.418055941307483,
 'mediana': 3.448,
 'minimo': 0.501}

In [ ]:
print(f'distancia máxima del viaje: {dist_km_max} km')

distancia máxima del viaje: 15.998 km


In [ ]:
print('duración del viaje en horas')
trip_duration = summary('trip_duration')
trip_duration_max = trip_duration['max']
trip_duration

duración del viaje en horas


{'IQR': 0.221,
 'Q1': 0.131,
 'Q2': 0,
 'Q3': 0.353,
 'max': 0.94,
 'meadi': 0.2616091677948092,
 'mediana': 0.219,
 'minimo': 0.014}

In [ ]:
print(f'duración máxima del viaje: {trip_duration_max*60} min')

duración máxima del viaje: 56.4 min


In [ ]:
print('tiempo de espera en segundos')
wait_sec_clean = summary('wait_sec')
wait_sec_clean_max = wait_sec_clean['max']
wait_sec_clean


tiempo de espera en segundos


{'IQR': 277.0,
 'Q1': 86.0,
 'Q2': 190,
 'Q3': 363.0,
 'max': 901,
 'meadi': 252.61194812000525,
 'mediana': 190.0,
 'minimo': 0}

In [ ]:
print(f'tiempo máximo de espera: {round(wait_sec_clean_max/60,3)} min')

tiempo máximo de espera: 15.017 min


In [ ]:
data_filtered.shape

(7633, 14)

### Resultados limpieza 

Con la limpieza mediante cuartiles, nos ayudo a detectar outlayers, obtuvimos mejores resultados.

- El valor máximo de duración del viaje es ~ 57 min.
- El valor máximo de distancia del viaje es ~ 16 km.
- El timpo máximo de espera del viaje es 15 min.
- La velocidad máxima del viaje es ~ 38.126 km/h

Estos valores ya son más utiles para trabajar con ellos.

# 4.Api GeoNames

OBJETIVO 

- Practicar el uso de APIs y peticiones HTTP.
- Automatizar procesos de exploración y limpieza mediante la creación de funciones

DESARROLLO

1. Utilización de la API [GeoNames](http://www.geonames.org/export/web-services.html#findNearbyPostalCodes)
2. Crear una cuenta si es necesario.
3. Leer la documentación.
4. Realizar algunas peticiones de prueba.
5. Automatiza el proceso de realizar peticiones para obtener un dataset considerablemente grande.
6. Explora y limpia tu dataset.





Conversión de coordenadas de el inicio y el fin de los viajes a el nómbre del municipio.
**Para realizar un análisis con respecto a las demarcaciones territoriales o municipios y los viaje es necesario localizar por las demarcaciones territoriales o municipios del inicio y fin de cada viaje.**

### Reverse Geocoding

```Reverse Geocoding``` es el proceso de encontrar una dirección, una calle, u alguna infromación a partir del par longitud/latitud. Esta tarea se puede llevar a cabo mediante
el uso de la Api GeoNames la cual ofrece un aplio rango de herramientas para esta tarea.
Para usar la Api, se necesita crear un *username*, sin embargo, las peticiones por hora están limitadas a 1000 por hora. Como se tienen que procesar cerca de 8000 datos, se dieron de alta 9 usuarios para alternar las peticiones y así recortar el tiempo del procesamiento de las longitudes y latitudes de los viajes.

#### Función de petición **GET**
Se define una función para hacer una petición tipo **GET** a la Api, esta función se usará junto con el método `apply`

**Ejemplo de petición a la Api**

In [ ]:
import requests as rq
import time

username="alanVillasana"
lat="19.434102"
lng="-99.085797"
maxRows="1"
style="MEDIUM"
radius="10"
url_api = "http://api.geonames.org/findNearbyPostalCodesJSON"

In [ ]:
endpoint = url_api+"?&username="+username+"&lat="+lat+"&lng="+lng+"&maxRows="+maxRows+"&style="+style+"&radius="+radius
endpoint

'http://api.geonames.org/findNearbyPostalCodesJSON?&username=alanVillasana&lat=19.434102&lng=-99.085797&maxRows=1&style=MEDIUM&radius=10'

Ejemplo del uso de GeoNames

In [ ]:
r = rq.get(endpoint)
result = r.json()['postalCodes'][0]
print (result["adminName2"])

Venustiano Carranza


In [ ]:

def get_location_by_coo (coordenates):
    import requests as rq
    import time
    global i
    coordenates = coordenates.split(",") #como las coordenadas se pasan concatenadas se tienen que separar
    lat = coordenates[0].strip()
    lng = coordenates[1].strip()
    usernames = ["beduProyect1","beduProyect2","beduProyect3","beduProyect4","beduProyect5","beduProyect6","alanVillasana","cristophercano","dgil"]
    # Debido a que la API solo deja hacer 1000 peticiones por usuario cada hora, se dieron de alta 9 usuarios para poder 
    # obtener el nombre del municipio o dearcacion territorial de la ZMVM en una sola ejecución. Ya que si usaramos solamente
    # un usuario de la API se tardaría aproximadamente 8 horas para poder completar las peticiones necesarias.
    username = usernames[i%9]
    maxRows="1"
    style="MEDIUM"
    radius="10"
    url = "http://api.geonames.org/findNearbyPostalCodesJSON"
    time.sleep(0.05)
    endpoint = url+"?&username="+username+"&lat="+lat+"&lng="+lng+"&maxRows="+maxRows+"&style="+style+"&radius="+radius
    r = rq.get(endpoint)
    print ("\r",i," ",username, end="")
    i = i+1
    result = r.json()['postalCodes'][0]
    

    return result['adminName2']

Se define la funcion  `get_location_by_name` para poder usar el método `aply` en el data frame.
Esta función hará una petició *GET* a la api de **GeoNames**, le enviará de parámetro las coordenadas de la locación y regresará el municiío o demarcación territorial correspondiente a las coordenadas.

#### Concatenación de latitud y longitud.
Se crearán dos columnas temporales que contendran el string concatenado de la latitud y longitud del inicio y final de los viajes.

In [ ]:
data_filtered["pickup_coordenates"] = data_filtered["pickup_latitude"].astype(str)+","+data_filtered["pickup_longitude"].astype(str)
data_filtered["dropoff_coordenates"] = data_filtered["dropoff_latitude"].astype(str)+","+data_filtered["dropoff_longitude"].astype(str)

In [ ]:
data_filtered["dropoff_coordenates"].head(2)

id
3    19.328530308412198,-99.2718736003419
4      19.3262558086244,-99.2798998325071
Name: dropoff_coordenates, dtype: object

#### Proceso de conversión de coordenadas
**Los bloques de código siguientes ocupan mucho tiempo de ejecución debido a la cantidad de llamadas que tiene que hacer a la API de GeoNames, por esto recomendamos que si se quiere ejecutar se haga con 1 hora de separación entre los dos bloques de código.**
\
Al final de esta sección se guardará un archivo csv con los resultados, para que no se tengan que ejecutar los bloques de código, solo leer el archivo.

In [ ]:
i = 0
data_filtered["pickup_location"] = data_filtered["pickup_coordenates"].apply(get_location_by_coo)

In [ ]:
i = 0
#data_filtered["dropoff_location"] = data_filtered["dropoff_coordenates"].apply(get_location_by_coo)

In [ ]:
data_filtered.sample(5)

,tipo_transporte,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,dist_meters,wait_sec,dist_km,speed_km,pickup_coordenates,dropoff_coordenates
id,,,,,,,,,,,,,,
11461,Taxi Libre,2017-07-05 10:21:22,2017-07-05 10:39:18,-99.196493,19.461894,-99.196294,19.461592,0.298889,7976,215,7.976,26.685502,"19.4618941211885,-99.196493024654","19.46159196,-99.19629416"
3978,Taxi Libre,2017-05-12 12:33:27,2017-05-12 12:55:39,-99.197505,19.461992,-99.195871,19.461522,0.370278,4829,538,4.829,13.041560,"19.4619924735365,-99.1975049787427","19.4615224476789,-99.1958710530103"
8684,Taxi Libre,2016-07-15 08:03:37,2016-07-15 08:10:52,-99.203432,19.380484,-99.207125,19.373369,0.121111,1540,89,1.540,12.715596,"19.3804843056338,-99.20343231382842","19.37336914,-99.20712498"
8813,Taxi Libre,2017-05-24 10:17:38,2017-05-24 10:22:19,-99.197337,19.461514,-99.185196,19.458702,0.078056,1738,56,1.738,22.266192,"19.461513851377198,-99.1973366350477","19.4587021517078,-99.1851960647948"
12309,Taxi de Sitio,2017-04-07 03:51:40,2017-04-07 04:01:46,-99.169860,19.281138,-99.160823,19.272655,0.168333,3331,125,3.331,19.788119,"19.2811377008547,-99.1698604781513","19.27265466,-99.16082266"


In [ ]:
#data_filtered = data_filtered.drop(columns = ["pickup_coordenates", "dropoff_coordenates"], axis = 1)
data_filtered.dtypes

tipo_transporte                object
pickup_datetime        datetime64[ns]
dropoff_datetime       datetime64[ns]
pickup_longitude              float64
pickup_latitude               float64
dropoff_longitude             float64
dropoff_latitude              float64
trip_duration                 float64
dist_meters                     int64
wait_sec                        int64
dist_km                       float64
speed_km                      float64
pickup_coordenates             object
dropoff_coordenates            object
dtype: object

In [ ]:
data_filtered.shape

(7633, 14)

## Eliminación de viajes fuera de la Zona Metropolitana del Valle de México
Se encontraron viajes que son fuera de la Zona Metropolitana del Valle de México *(ZMVM)*, como este análisis es solamente de la ZMVM se eliminarán los viajes que se realizaron fuera de esta.

Se define una lista (`ZMVM`) con los municipios y demarcaciones territoriales que conforman la **Zona Metropolitana del Valle de México.** 

In [ ]:
ZMVM = ['Acolman',
 'Atizapán de Zaragoza',
 'Azcapotzalco',
 'Benito Juárez',
 'Chalco',
 'Chimalhuacán',
 'Coacalco de Berriozábal',
 'Cocotitlán',
 'Coyoacán',
 'Cuajimalpa de Morelos',
 'Cuauhtémoc',
 'Cuautitlán',
 'Cuautitlán Izcalli',
 'Ecatepec de Morelos',
 'Gustavo A. Madero',
 'Huixquilucan',
 'Ixtapaluca',
 'Iztacalco',
 'Iztapalapa',
 'La Magdalena Contreras',
 'La Paz',
 'Miguel Hidalgo',
 'Milpa Alta',
 'Naucalpan de Juárez',
 'Nezahualcóyotl',
 'Tecámac',
 'Tizayuca',
 'Tlalnepantla de Baz',
 'Tlalpan',
 'Tláhuac',
 'Tultepec',
 'Tultitlán',
 'Valle de Chalco Solidaridad',
 'Venustiano Carranza',
 'Xochimilco',
 'Álvaro Obregón']

In [ ]:
#data_locaciones = data_filtered[(data_filtered["pickup_location"].isin(ZMVM))
#                            & (data_filtered["dropoff_location"].isin (ZMVM))]

In [ ]:
#data_locaciones.shape

### Exportación de DataFrame resultante.

In [ ]:
#data_locaciones.to_csv("cdmx_transporte_location.csv")

In [ ]:
url = 'https://raw.githubusercontent.com/CristopherCano/Proyecto_Python_Procesamiento_de_datos/main/Data/cdmx_transporte_location.csv'
data_municipios = pd.read_csv(url)
data_municipios['pickup_datetime'] = pd.to_datetime(data_municipios['pickup_datetime'], unit = 'ns')
data_municipios['dropoff_datetime'] = pd.to_datetime(data_municipios['dropoff_datetime'], unit = 'ns')
data_municipios.head()

,id,tipo_transporte,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,dist_meters,wait_sec,dist_km,speed_km,pickup_location,dropoff_location
0,3,Taxi Libre,2016-09-18 10:11:50,2016-09-18 10:23:11,-99.289603,19.326263,-99.271874,19.328530,0.189167,2848,129,2.848,15.055507,Cuajimalpa de Morelos,Álvaro Obregón
1,4,Taxi Libre,2016-09-18 10:23:38,2016-09-18 10:30:53,-99.271161,19.328875,-99.279900,19.326256,0.121111,1409,106,1.409,11.633945,Álvaro Obregón,Álvaro Obregón
2,5,Taxi Libre,2016-09-18 10:44:18,2016-09-18 10:51:40,-99.282761,19.326944,-99.291705,19.322754,0.122778,1567,85,1.567,12.762896,Álvaro Obregón,Cuajimalpa de Morelos
3,6,Taxi Libre,2016-09-18 10:58:07,2016-09-18 10:59:46,-99.289712,19.322929,-99.289280,19.325978,0.027778,797,19,0.797,28.692000,Cuajimalpa de Morelos,Cuajimalpa de Morelos
4,7,Taxi Libre,2016-09-18 11:00:00,2016-09-18 11:05:45,-99.285391,19.326247,-99.282549,19.328117,0.095833,676,169,0.676,7.053913,Álvaro Obregón,Álvaro Obregón


In [ ]:
data_municipios = data_municipios.set_index(keys = ["id"])
data_municipios.shape

(7626, 14)

# 5.Tarifas y Costos

### Tarifas

**Descripción**

El archivo ```tarifas.json``` contiene los datos de tarifas relacionadas al desglose del costo total de un viaje realizado dentro del rango de fechas utilizadas en la base de datos principal (2016-2017).
Las columnas son las siguientes

- ```Transporte```: Tipo de Taxi o Uber 
- ```banderazo```: Tarifa inicial del tipo de transporte.
- ```tarifa_dist```: Tarifa que se agrega cada cierta distancia avanzada. (250mts. para Taxis, 1km. para Uber)
- ```tarifa_ tiempo```: Tarifa que se agrega cada cierto tiempo transcurrido. (45seg. para Taxis, 1min. para Uber)
- ```tarifa_min```: Costo final que será aplicado en caso de que el monto calculado no supere esta cantidad.

Las tarifas relacionadas a los Taxis están reguladas por la Secretaría de Movilidad de la Ciudad de México, éstas se publican regularmente en la Gaceta Oficial, que es el órgano del Gobierno de la Ciudad de México, que tiene como finalidad publicar todas aquellas disposiciones emanadas de autoridad competente que tengan aplicación en el ámbito de la CDMX y están disponibles para su consulta en Internet.

Las tarifas de Uber se consiguieron al consultar múltiples páginas de Internet que brindaban información acerca de ello, se hizo una comparación general de toda la información y se seleccionó aquella que concordara con la mayoría. En su app y sitio web, Uber muestra el desglose de tarifas, sin embargo, éstas son actuales y no se adaptan a las fechas de nuestra base de datos. También se intentó acceder a una API para la consulta de costos, pero a pesar de generar una cuenta, no se contaban con los permisos necesarios para las consultas de información.

In [ ]:
#Link del archivo JSON en el repositorio
url_tarifas = "https://raw.githubusercontent.com/CristopherCano/Proyecto_Python_Procesamiento_de_datos/main/Data/tarifas.json"

#Lectura de archivo JSON a DataFrame
tarifas = pd.read_json(url_tarifas)

tarifas

,Transporte,Banderazo,Tarifa Distancia,Tarifa Tiempo,Tarifa Mínima
0,Taxi Libre,8.74,1.07,1.07,8.74
1,Taxi de Sitio,13.10,1.30,1.30,13.10
2,Radio Taxi,27.60,1.84,1.84,27.60
3,UberX,7.00,3.57,1.80,35.00
4,UberXL,12.15,6.28,3.15,45.50
5,UberBlack,30.00,9.46,3.50,77.00
6,UberSUV,40.00,15.00,4.00,150.00


In [ ]:
#Se revisa que las variables tengan los formatos correctos
tarifas.dtypes

Transporte           object
Banderazo           float64
Tarifa Distancia    float64
Tarifa Tiempo       float64
Tarifa Mínima       float64
dtype: object

In [ ]:
#Se corrigen los nombres de las columnas para seguir la convención snake_case
tarifas.columns = ["transporte", "banderazo", "tarifa_dist", "tarifa_tiempo", "tarifa_min"]

tarifas

,transporte,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min
0,Taxi Libre,8.74,1.07,1.07,8.74
1,Taxi de Sitio,13.10,1.30,1.30,13.10
2,Radio Taxi,27.60,1.84,1.84,27.60
3,UberX,7.00,3.57,1.80,35.00
4,UberXL,12.15,6.28,3.15,45.50
5,UberBlack,30.00,9.46,3.50,77.00
6,UberSUV,40.00,15.00,4.00,150.00


In [ ]:
#Se crea un subDataFrame que contenga las columnas de interés para calcular costos del viaje
viajes = data_municipios[["tipo_transporte", "wait_sec", "dist_meters", "pickup_datetime"]]

#Se une el subDataFrame con el JSON en un mismo DataFrame
#La variable en común será "tipo_transporte" y "transporte" que contienen las categorías de Taxi y Uber
#how="left" es para mantener todas las filas del subDataFrame y se trabajen sobre ellas
costos_viajes = pd.merge(viajes, tarifas, left_on="tipo_transporte", right_on="transporte", how="left")

#Se elimina la columna transporte ya que es la misma que "tipo_transporte"
costos_viajes = costos_viajes.drop(columns=["transporte"])

costos_viajes

,tipo_transporte,wait_sec,dist_meters,pickup_datetime,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min
0,Taxi Libre,129,2848,2016-09-18 10:11:50,8.74,1.07,1.07,8.74
1,Taxi Libre,106,1409,2016-09-18 10:23:38,8.74,1.07,1.07,8.74
2,Taxi Libre,85,1567,2016-09-18 10:44:18,8.74,1.07,1.07,8.74
3,Taxi Libre,19,797,2016-09-18 10:58:07,8.74,1.07,1.07,8.74
4,Taxi Libre,169,676,2016-09-18 11:00:00,8.74,1.07,1.07,8.74
...,...,...,...,...,...,...,...,...
7621,Taxi de Sitio,150,4908,2016-10-27 04:52:10,13.10,1.30,1.30,13.10
7622,Taxi Libre,386,7407,2016-10-27 05:02:26,8.74,1.07,1.07,8.74
7623,Taxi Libre,33,2280,2016-10-27 08:11:10,8.74,1.07,1.07,8.74
7624,Taxi de Sitio,115,2498,2016-10-27 10:53:15,13.10,1.30,1.30,13.10


### Costos

Se calculan los costos totales de cada viaje. 

Formula para calcular el Costo Total del viaje:


$Costo Viaje = TarifaBase + (Tarifa Distancia * Distancia Recorrida) + (Tarifa Tiempo * Tiempo Espera )$


Es importante recordar que cada categoría Uber/Taxi maneja diferentes unidades de distancia y de tiempo. 

Para Uber es 1km y 1min y para Taxi son 250mts. y 45seg. por lo que se debe de dividir la distancia y el tiempo de espera total para obtener las unidades correctas

*Tiempo Espera se entiende como el tiempo en el que el automóvil estuvo detenido, por ejemplo semáforo o tráfico.*


In [ ]:
#Las filas que contengan los valores de la cateogría de Taxi en "tipo_transporte" utilizarán los valores pertinentes
#para el correcto cálculo del Costo Viaje
costos_viajes.loc[(costos_viajes["tipo_transporte"] == "Taxi Libre") |
                  (costos_viajes["tipo_transporte"] == "Taxi de Sitio") |
                  (costos_viajes["tipo_transporte"] == "Radio Taxi"),
                  "costo_viaje_sin_min"] = (costos_viajes["banderazo"] + 
                                ((costos_viajes["dist_meters"] / 250) * costos_viajes["tarifa_dist"]) + 
                                ((costos_viajes["wait_sec"] / 45) * costos_viajes["tarifa_tiempo"]))

#Las filas que contengan los valores de la cateogría de Uber en "tipo_transporte" utilizarán los valores pertinentes
#para el correcto cálculo del Costo Viaje
costos_viajes.loc[(costos_viajes["tipo_transporte"] == "UberX") |
                  (costos_viajes["tipo_transporte"] == "UberXL") |
                  (costos_viajes["tipo_transporte"] == "UberBlack") |
                  (costos_viajes["tipo_transporte"] == "UberSUV"),
                  "costo_viaje_sin_min"] = (costos_viajes["banderazo"] + 
                                ((costos_viajes["dist_meters"] / 1000) * costos_viajes["tarifa_dist"]) + 
                                ((costos_viajes["wait_sec"] / 60) * costos_viajes["tarifa_tiempo"]))

costos_viajes

#Con los valores arrojados, se puede corroborar que las nuevas columnas son correctas

,tipo_transporte,wait_sec,dist_meters,pickup_datetime,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min,costo_viaje_sin_min
0,Taxi Libre,129,2848,2016-09-18 10:11:50,8.74,1.07,1.07,8.74,23.996773
1,Taxi Libre,106,1409,2016-09-18 10:23:38,8.74,1.07,1.07,8.74,17.290964
2,Taxi Libre,85,1567,2016-09-18 10:44:18,8.74,1.07,1.07,8.74,17.467871
3,Taxi Libre,19,797,2016-09-18 10:58:07,8.74,1.07,1.07,8.74,12.602938
4,Taxi Libre,169,676,2016-09-18 11:00:00,8.74,1.07,1.07,8.74,15.651724
...,...,...,...,...,...,...,...,...,...
7621,Taxi de Sitio,150,4908,2016-10-27 04:52:10,13.10,1.30,1.30,13.10,42.954933
7622,Taxi Libre,386,7407,2016-10-27 05:02:26,8.74,1.07,1.07,8.74,49.620182
7623,Taxi Libre,33,2280,2016-10-27 08:11:10,8.74,1.07,1.07,8.74,19.283067
7624,Taxi de Sitio,115,2498,2016-10-27 10:53:15,13.10,1.30,1.30,13.10,29.411822


### Aplicación de Tarifa Mínima

Sin importar que la distancia o el tiempo recorrido sean muy pequeños, existe una Tarifa Mínima para que esta sea aplicada. 

Es decir, si el costo total del viaje era de \$5 y la tarifa mínima es de \$20, el monto que deberá pagar el pasajero será de $20.


In [ ]:
#Se duplica la columna del Costo Viaje calculado
costos_viajes["costo_viaje_sobre_min"] = costos_viajes["costo_viaje_sin_min"]

#En la columna duplicada se aplica código
#Si el Costo Viaje es menor a la Tarifa Mínima, Tarifa Mínima sustituye el valor de esa fila en la nueva columna
costos_viajes.loc[costos_viajes["costo_viaje_sobre_min"] < costos_viajes["tarifa_min"], "costo_viaje_sobre_min"] = costos_viajes["tarifa_min"]

costos_viajes

#Con los valores arrojados, se puede corroborar que las nuevas columnas son correctas

,tipo_transporte,wait_sec,dist_meters,pickup_datetime,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min,costo_viaje_sin_min,costo_viaje_sobre_min
0,Taxi Libre,129,2848,2016-09-18 10:11:50,8.74,1.07,1.07,8.74,23.996773,23.996773
1,Taxi Libre,106,1409,2016-09-18 10:23:38,8.74,1.07,1.07,8.74,17.290964,17.290964
2,Taxi Libre,85,1567,2016-09-18 10:44:18,8.74,1.07,1.07,8.74,17.467871,17.467871
3,Taxi Libre,19,797,2016-09-18 10:58:07,8.74,1.07,1.07,8.74,12.602938,12.602938
4,Taxi Libre,169,676,2016-09-18 11:00:00,8.74,1.07,1.07,8.74,15.651724,15.651724
...,...,...,...,...,...,...,...,...,...,...
7621,Taxi de Sitio,150,4908,2016-10-27 04:52:10,13.10,1.30,1.30,13.10,42.954933,42.954933
7622,Taxi Libre,386,7407,2016-10-27 05:02:26,8.74,1.07,1.07,8.74,49.620182,49.620182
7623,Taxi Libre,33,2280,2016-10-27 08:11:10,8.74,1.07,1.07,8.74,19.283067,19.283067
7624,Taxi de Sitio,115,2498,2016-10-27 10:53:15,13.10,1.30,1.30,13.10,29.411822,29.411822


### Agrupaciones por mes, días y horas

Para poder obtener algunos estadísticos se separa la fecha en Hora, Día y Mes

Con ésto, podremos obtener el promedio por categoría (Uber/Taxi) por Hora del día, Día del Mes o Mes del Año. De esta manera podemos saber a qué Hora/Día/Mes le conviene trabajar al conductor.

#### Separación de Fecha en Hora, Día, Mes

In [ ]:
#Se obtiene la columna Hora
costos_viajes["hora"] = costos_viajes["pickup_datetime"].apply(lambda x: x.strftime("%I"))

#Se obtiene la columna Día
costos_viajes["dia"] = costos_viajes["pickup_datetime"].apply(lambda x: x.strftime("%A"))

#Se obtiene la columna Mes
costos_viajes["mes"] = costos_viajes["pickup_datetime"].apply(lambda x: x.strftime("%B"))

costos_viajes

#Con los valores arrojados, se puede corroborar que las nuevas columnas son correctas

,tipo_transporte,wait_sec,dist_meters,pickup_datetime,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min,costo_viaje_sin_min,costo_viaje_sobre_min,hora,dia,mes
0,Taxi Libre,129,2848,2016-09-18 10:11:50,8.74,1.07,1.07,8.74,23.996773,23.996773,10,Sunday,September
1,Taxi Libre,106,1409,2016-09-18 10:23:38,8.74,1.07,1.07,8.74,17.290964,17.290964,10,Sunday,September
2,Taxi Libre,85,1567,2016-09-18 10:44:18,8.74,1.07,1.07,8.74,17.467871,17.467871,10,Sunday,September
3,Taxi Libre,19,797,2016-09-18 10:58:07,8.74,1.07,1.07,8.74,12.602938,12.602938,10,Sunday,September
4,Taxi Libre,169,676,2016-09-18 11:00:00,8.74,1.07,1.07,8.74,15.651724,15.651724,11,Sunday,September
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7621,Taxi de Sitio,150,4908,2016-10-27 04:52:10,13.10,1.30,1.30,13.10,42.954933,42.954933,04,Thursday,October
7622,Taxi Libre,386,7407,2016-10-27 05:02:26,8.74,1.07,1.07,8.74,49.620182,49.620182,05,Thursday,October
7623,Taxi Libre,33,2280,2016-10-27 08:11:10,8.74,1.07,1.07,8.74,19.283067,19.283067,08,Thursday,October
7624,Taxi de Sitio,115,2498,2016-10-27 10:53:15,13.10,1.30,1.30,13.10,29.411822,29.411822,10,Thursday,October


#### Resultados

In [ ]:
#Se agrupa por categoría (Uber/Taxi) y se obtiene el promedio del Costo Viaje
costos_viajes.groupby("tipo_transporte")["costo_viaje_sobre_min"].mean()

tipo_transporte
Radio Taxi        80.020915
Taxi Libre        31.785366
Taxi de Sitio     46.209170
UberBlack         89.169905
UberSUV          159.087059
UberX             39.451198
UberXL            70.983620
Name: costo_viaje_sobre_min, dtype: float64

* Si se comparan Taxi Libre y UberX que son su equivalencia en su categoría, se observa que el costo es menor en Taxi Libre

* En este caso, si la información es para un conductor de transporte, a él le convendría trabajar para Uber. Sin emabrgo, también se deben tomar en cuenta las comisiones que van dirigidas hacia la aplicación.

* Si la información es para un usuario de transporte, le conviene usar Taxi Libre ya que pagará menos.

* Estos precios son promedio y son sólo aproximaciones. Como se puede observar en la fórmula para obtener el Costo Viaje dependen de las variables de la distancia recorrida y el tiempo de espera.

In [ ]:
#Se agrupa por categoría (Uber/Taxi) y Hora del viaje y se obtiene el promedio del Costo Viaje
var = costos_viajes.groupby(["tipo_transporte","hora"], as_index=False)["costo_viaje_sobre_min"].mean()

#Código para imprimir el resultado completo y no sólo el principio y final de las filas
pd.set_option('display.max_rows', var.shape[0]+1)

#Se imprimen los valores realizando un sort. 
#El primer valor por categoría contiene el promedio más alto
#Por lo tanto conviene trabajar a esa hora
var.sort_values(["tipo_transporte", "costo_viaje_sobre_min"], ascending=[True, False])

,tipo_transporte,hora,costo_viaje_sobre_min
10,Radio Taxi,11,86.518513
8,Radio Taxi,09,85.423986
5,Radio Taxi,06,85.160701
7,Radio Taxi,08,84.501886
11,Radio Taxi,12,82.566377
4,Radio Taxi,05,80.646574
0,Radio Taxi,01,79.095401
9,Radio Taxi,10,77.742263
1,Radio Taxi,02,77.617503
6,Radio Taxi,07,73.762984


In [ ]:
#Se agrupa por categoría (Uber/Taxi) y Día del viaje y se obtiene el promedio del Costo Viaje
var = costos_viajes.groupby(["tipo_transporte","dia"], as_index=False)["costo_viaje_sobre_min"].mean()

#Se imprimen los valores realizando un sort. 
#El primer valor por categoría contiene el promedio más alto
#Por lo tanto conviene trabajar en ese día
var.sort_values(["tipo_transporte", "costo_viaje_sobre_min"], ascending=[True, False])

,tipo_transporte,dia,costo_viaje_sobre_min
2,Radio Taxi,Saturday,84.871787
4,Radio Taxi,Thursday,83.143862
3,Radio Taxi,Sunday,80.395321
1,Radio Taxi,Monday,79.783677
0,Radio Taxi,Friday,79.162651
5,Radio Taxi,Tuesday,76.756291
6,Radio Taxi,Wednesday,76.006537
10,Taxi Libre,Sunday,33.477931
9,Taxi Libre,Saturday,32.863114
11,Taxi Libre,Thursday,32.341755


In [ ]:
#Se agrupa por categoría (Uber/Taxi) y Mes del viaje y se obtiene el promedio del Costo Viaje
var = costos_viajes.groupby(["tipo_transporte","mes"], as_index=False)["costo_viaje_sobre_min"].mean()

##Se imprimen los valores realizando un sort. 
#El primer valor por categoría contiene el promedio más alto
#Por lo tanto conviene trabajar en ese mes
var.sort_values(["tipo_transporte", "costo_viaje_sobre_min"], ascending=[True, False])

,tipo_transporte,mes,costo_viaje_sobre_min
4,Radio Taxi,January,90.719291
11,Radio Taxi,September,85.535372
2,Radio Taxi,December,84.648883
1,Radio Taxi,August,83.926571
3,Radio Taxi,February,83.721159
9,Radio Taxi,November,82.262140
7,Radio Taxi,March,81.953751
5,Radio Taxi,July,80.319438
10,Radio Taxi,October,77.673058
6,Radio Taxi,June,72.566968


In [ ]:
new_columns = ["banderazo","tarifa_dist","tarifa_tiempo","tarifa_min","costo_viaje_sin_min","costo_viaje_sobre_min"]

In [ ]:
data_costos = pd.concat([data_filtered.reset_index(),costos_viajes[new_columns].reset_index()], axis = 1)
data_costos = data_costos.drop(columns = ["index"], axis = 1).set_index("id")

In [ ]:
data_costos.dtypes

tipo_transporte                  object
pickup_datetime          datetime64[ns]
dropoff_datetime         datetime64[ns]
pickup_longitude                float64
pickup_latitude                 float64
dropoff_longitude               float64
dropoff_latitude                float64
trip_duration                   float64
dist_meters                       int64
wait_sec                          int64
dist_km                         float64
speed_km                        float64
pickup_coordenates               object
dropoff_coordenates              object
banderazo                       float64
tarifa_dist                     float64
tarifa_tiempo                   float64
tarifa_min                      float64
costo_viaje_sin_min             float64
costo_viaje_sobre_min           float64
dtype: object

In [ ]:
data_costos.shape

(7633, 20)

El data frame `data_costos` ha pasado por el proceso de limpieza y procesamiento de algunas columnas, se considera que está listo para cualquier análisis o modelado que se requiera hacer.
El data frame se guardará en un archivo *csv* para su utilización en otros proyectos.

In [ ]:
data_costos.to_csv("cdmx_transporte.csv")

# 6.Análisis Exploratorio

OBJETIVO

- Realizar filtraciones para obtener conjuntos de datos.
- Reordenar datos para ver el conjunto desde diferentes perspectivas.

DESARROLLO

Ha llegado el momento de aplicar técnicas comunes de procesamiento de datos a nuestro conjunto de datos. Para este momento nuestro conjunto de datos ya ha sido limpiado previamente de `NaNs`. Se realizo previamente un exploración extensivamente del dataset y se conocen su información. El dataset esté bien indexado y tiene nombres coherentes y comprensibles para las columnas.

Para aplicar en nuestro dataset lo que vimos en esta las ultimas sesiones, realizamos las siguientes acciones:

1. Checa que todos tus datos tengan el tipo de dato correcto. Si no es así, usa casting para convertir tus datos al tipo de dato correcto (recuerda que tipos de dato como `datetime64` se guardan como strings cuando están en archivos .csv, así que tendrás que convertirlos al tipo de dato apropiado cada vez que importes tu archivo.)
2. Si tienes columnas de texto, asegúrate de que todas tengan el formato correcto. Si no es así, utiliza las técnicas de manipulación de `strings` para darles el formato que necesitas.
3. Si consideras que alguna de tus columnas sería más clara si los datos tuvieran otro formato o representación usa `map` para transformar los datos de esa columna.
4. Si crees que es posible generar nuevas columnas útiles a partir de las columnas que ya tienes, usa `apply` para generar nuevos datos a partir de los que tienes y añádelos a tu dataset.
5. Con el fin de responder algunas de las preguntas que te planteaste acerca de tu dataset, usa filtros y sorting para crear nuevos subconjuntos y reordenamientos que sean más adecuados para responder tus preguntas. Primero comienza intentando responder las preguntas que te planteaste al principio, pero después puedes solamente explorar para ver si encuentras otras preguntas que no te habías planteado anteriormente.


In [ ]:
#mun = data_costos.copy()
url = 'https://raw.githubusercontent.com/CristopherCano/Proyecto_Python_Procesamiento_de_datos/main/Data/cdmx_transporte.csv'
mun = pd.read_csv(url, index_col=0)
mun.head()

,tipo_transporte,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,dist_meters,wait_sec,dist_km,speed_km,pickup_location,dropoff_location,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min,costo_viaje_sin_min,costo_viaje_sobre_min
id,,,,,,,,,,,,,,,,,,,,
3,Taxi Libre,2016-09-18 10:11:50,2016-09-18 10:23:11,-99.289603,19.326263,-99.271874,19.328530,0.189167,2848,129,2.848,15.055507,Cuajimalpa de Morelos,Álvaro Obregón,8.74,1.07,1.07,8.74,23.996773,23.996773
4,Taxi Libre,2016-09-18 10:23:38,2016-09-18 10:30:53,-99.271161,19.328875,-99.279900,19.326256,0.121111,1409,106,1.409,11.633945,Álvaro Obregón,Álvaro Obregón,8.74,1.07,1.07,8.74,17.290964,17.290964
5,Taxi Libre,2016-09-18 10:44:18,2016-09-18 10:51:40,-99.282761,19.326944,-99.291705,19.322754,0.122778,1567,85,1.567,12.762896,Álvaro Obregón,Cuajimalpa de Morelos,8.74,1.07,1.07,8.74,17.467871,17.467871
6,Taxi Libre,2016-09-18 10:58:07,2016-09-18 10:59:46,-99.289712,19.322929,-99.289280,19.325978,0.027778,797,19,0.797,28.692000,Cuajimalpa de Morelos,Cuajimalpa de Morelos,8.74,1.07,1.07,8.74,12.602938,12.602938
7,Taxi Libre,2016-09-18 11:00:00,2016-09-18 11:05:45,-99.285391,19.326247,-99.282549,19.328117,0.095833,676,169,0.676,7.053913,Álvaro Obregón,Álvaro Obregón,8.74,1.07,1.07,8.74,15.651724,15.651724


### Clasificando los viajes

In [ ]:
def summary(nombre):
  minimo = mun[nombre].min()
  Q1 = mun[nombre].quantile(0.25)
  median = mun[nombre].median()
  mean = mun[nombre].mean()
  Q2 = mun[nombre].quantile(0.5)
  Q3 = mun[nombre].quantile(0.75)
  maximo = mun[nombre].max()
  IQR = Q3 - Q1
  resumen = {'minimo': minimo, 'Q1': Q1, 'mediana':median,'Q2':Q2,'Q3':Q3,'max':maximo,'IQR':IQR}
  return resumen


In [ ]:
cuartiles_dist = summary('dist_km')
cuartiles_dist

{'IQR': 3.9910000000000005,
 'Q1': 1.97,
 'Q2': 3.448,
 'Q3': 5.961,
 'max': 15.998,
 'mediana': 3.448,
 'minimo': 0.501}

In [ ]:
def classificar_viajes(distancia, cuartiles):
  x = distancia
  Q1 = cuartiles['Q1']
  Q3 = cuartiles['Q2']

  if x < Q1:
    resultado = 'corto'
    return resultado
  elif Q1 < x < Q3:
    resultado = 'mediano'
    return resultado
  elif Q3 < x:
    resultado = 'largo'
    return resultado

In [ ]:
dist_km_col = mun['dist_km']

In [ ]:
dist_km_col.apply(classificar_viajes, args=(cuartiles_dist,))

id
3        mediano
4          corto
5          corto
6          corto
7          corto
          ...   
12686      largo
12687      largo
12688    mediano
12691    mediano
12692      largo
Name: dist_km, Length: 7633, dtype: object

In [ ]:
mun['viaje'] = dist_km_col.apply(classificar_viajes, args=(cuartiles_dist,))
mun.head()

,tipo_transporte,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,dist_meters,wait_sec,dist_km,speed_km,pickup_location,dropoff_location,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min,costo_viaje_sin_min,costo_viaje_sobre_min,viaje
id,,,,,,,,,,,,,,,,,,,,,
3,Taxi Libre,2016-09-18 10:11:50,2016-09-18 10:23:11,-99.289603,19.326263,-99.271874,19.328530,0.189167,2848,129,2.848,15.055507,Cuajimalpa de Morelos,Álvaro Obregón,8.74,1.07,1.07,8.74,23.996773,23.996773,mediano
4,Taxi Libre,2016-09-18 10:23:38,2016-09-18 10:30:53,-99.271161,19.328875,-99.279900,19.326256,0.121111,1409,106,1.409,11.633945,Álvaro Obregón,Álvaro Obregón,8.74,1.07,1.07,8.74,17.290964,17.290964,corto
5,Taxi Libre,2016-09-18 10:44:18,2016-09-18 10:51:40,-99.282761,19.326944,-99.291705,19.322754,0.122778,1567,85,1.567,12.762896,Álvaro Obregón,Cuajimalpa de Morelos,8.74,1.07,1.07,8.74,17.467871,17.467871,corto
6,Taxi Libre,2016-09-18 10:58:07,2016-09-18 10:59:46,-99.289712,19.322929,-99.289280,19.325978,0.027778,797,19,0.797,28.692000,Cuajimalpa de Morelos,Cuajimalpa de Morelos,8.74,1.07,1.07,8.74,12.602938,12.602938,corto
7,Taxi Libre,2016-09-18 11:00:00,2016-09-18 11:05:45,-99.285391,19.326247,-99.282549,19.328117,0.095833,676,169,0.676,7.053913,Álvaro Obregón,Álvaro Obregón,8.74,1.07,1.07,8.74,15.651724,15.651724,corto


Moda de los viajes por tipo de transporte y viaje

In [ ]:
df_viajes_o = mun.groupby(['tipo_transporte','viaje'])['pickup_location'].value_counts().rename('suma')
df_viajes_o = df_viajes_o.reset_index()
df_viajes_o.sort_values('suma', ascending=False)

,tipo_transporte,viaje,pickup_location,suma
65,Taxi Libre,largo,Iztapalapa,587
66,Taxi Libre,largo,Álvaro Obregón,342
93,Taxi Libre,mediano,Iztapalapa,333
67,Taxi Libre,largo,Miguel Hidalgo,319
13,Radio Taxi,largo,Benito Juárez,283
...,...,...,...,...
176,UberBlack,largo,Querétaro,1
175,UberBlack,corto,Miguel Hidalgo,1
174,UberBlack,corto,Benito Juárez,1
173,Taxi de Sitio,mediano,Tláhuac,1


**Para la distancia en km** \
viaje corto:     x < primer cuartil \
viaje mediano:   primer cuartil < x < tercer cuartil \
viaje largo:     tercer cuartil < x

In [ ]:
cuartiles_dist

{'IQR': 3.9910000000000005,
 'Q1': 1.97,
 'Q2': 3.448,
 'Q3': 5.961,
 'max': 15.998,
 'mediana': 3.448,
 'minimo': 0.501}

In [ ]:
df_viajes_o.groupby('pickup_location')[['viaje','tipo_transporte']].agg(pd.Series.mode)

,viaje,tipo_transporte
pickup_location,,
Acolman,largo,UberX
Ahome,largo,Taxi de Sitio
Atizapán de Zaragoza,"[largo, mediano]","[Taxi de Sitio, UberX]"
Azcapotzalco,mediano,"[Radio Taxi, Taxi Libre, Taxi de Sitio]"
Benito Juárez,"[corto, largo, mediano]","[Radio Taxi, Taxi Libre, Taxi de Sitio, UberX]"
Chalco,largo,"[Radio Taxi, Taxi Libre]"
Chimalhuacán,largo,Taxi Libre
Coacalco de Berriozábal,corto,Taxi Libre
Coyoacán,largo,"[Radio Taxi, Taxi Libre, Taxi de Sitio, UberX]"


### Viajes por origen y destino

In [ ]:
viajes = pd.melt(mun[['pickup_location','dropoff_location']])
viajes = viajes.groupby('variable')['value'].value_counts().rename('conteo')
viajes = pd.DataFrame(viajes)
viajes

conteo
variable         value                              
dropoff_location Iztapalapa                     1084
                 Miguel Hidalgo                 1018
                 Álvaro Obregón                  995
                 Cuauhtémoc                      730
                 Benito Juárez                   658
                 Tlalpan                         487
                 Coyoacán                        409
                 Gustavo A. Madero               358
                 La Magdalena Contreras          346
                 Tláhuac                         287
                 Azcapotzalco                    277
                 Venustiano Carranza             242
                 Xochimilco                      229
                 Iztacalco                       129
                 Cuajimalpa de Morelos           101
                 Ecatepec de Morelos              86
                 Nezahualcóyotl                   53
                 Milpa Alta                       25
                 Naucalpan de Juárez              25
                 Tlalnepantla de Baz              25
                 Coacalco de Berriozábal          20
                 Tecámac                          14
                 Tultitlán                         6
                 Chalco                            5
                 Ixtapaluca                        5
                 Huixquilucan                      4
                 Mérida                            3
                 Atizapán de Zaragoza              2
                 La Paz                            2
                 Querétaro                         2
                 Tultepec                          2
                 Ahome                             1
                 Cuautitlán Izcalli                1
                 Tulancingo de Bravo               1
                 Valle de Chalco Solidaridad       1
pickup_location  Iztapalapa                     1362
                 Miguel Hidalgo                 1241
                 Álvaro Obregón                 1026
                 Benito Juárez                   717
                 Cuauhtémoc                      508
                 Tlalpan                         483
                 Venustiano Carranza             457
                 Gustavo A. Madero               388
                 Coyoacán                        356
                 La Magdalena Contreras          356
                 Xochimilco                      214
                 Azcapotzalco                     99
                 Ecatepec de Morelos              92
                 Cuajimalpa de Morelos            76
                 Iztacalco                        58
                 Tláhuac                          47
                 Nezahualcóyotl                   39
                 Milpa Alta                       23
                 Tlalnepantla de Baz              22
                 Coacalco de Berriozábal          18
                 Tecámac                          13
                 Naucalpan de Juárez               7
                 Tultitlán                         5
                 Chalco                            4
                 Ixtapaluca                        3
                 Mérida                            3
                 Atizapán de Zaragoza              2
                 La Paz                            2
                 Querétaro                         2
                 Valle de Chalco Solidaridad       2
                 Acolman                           1
                 Ahome                             1
                 Chimalhuacán                      1
                 Cuautitlán                        1
                 Cuautitlán Izcalli                1
                 Huixquilucan                      1
                 Tulancingo de Bravo               1
                 Tultepec                          1

### Tiempo de espera la misma alcaldia

In [ ]:
mun_wait = mun[['pickup_location','pickup_longitude','pickup_latitude','dropoff_location','dropoff_longitude','dropoff_latitude','wait_sec']]
mun_wait.head()

,pickup_location,pickup_longitude,pickup_latitude,dropoff_location,dropoff_longitude,dropoff_latitude,wait_sec
id,,,,,,,
3,Cuajimalpa de Morelos,-99.289603,19.326263,Álvaro Obregón,-99.271874,19.328530,129
4,Álvaro Obregón,-99.271161,19.328875,Álvaro Obregón,-99.279900,19.326256,106
5,Álvaro Obregón,-99.282761,19.326944,Cuajimalpa de Morelos,-99.291705,19.322754,85
6,Cuajimalpa de Morelos,-99.289712,19.322929,Cuajimalpa de Morelos,-99.289280,19.325978,19
7,Álvaro Obregón,-99.285391,19.326247,Álvaro Obregón,-99.282549,19.328117,169


In [ ]:
mun_wait.shape

(7633, 7)

In [ ]:
mun_same = mun_wait[mun_wait['pickup_location'] == mun_wait['dropoff_location']]
mun_same.head()

,pickup_location,pickup_longitude,pickup_latitude,dropoff_location,dropoff_longitude,dropoff_latitude,wait_sec
id,,,,,,,
4,Álvaro Obregón,-99.271161,19.328875,Álvaro Obregón,-99.279900,19.326256,106
6,Cuajimalpa de Morelos,-99.289712,19.322929,Cuajimalpa de Morelos,-99.289280,19.325978,19
7,Álvaro Obregón,-99.285391,19.326247,Álvaro Obregón,-99.282549,19.328117,169
12,Cuajimalpa de Morelos,-99.290340,19.331214,Cuajimalpa de Morelos,-99.294896,19.320687,33
13,Miguel Hidalgo,-99.179629,19.438589,Miguel Hidalgo,-99.179107,19.438777,181


In [ ]:
mun_same.shape

(4808, 7)

In [ ]:
viajes_mean_wait_sec = mun_same.groupby('pickup_location')['wait_sec'].agg(['mean']).sort_values('pickup_location')
viajes_misma_locacion = pd.DataFrame(mun_same.value_counts('pickup_location').rename('conteo')).sort_values('pickup_location')

Tiempo de espera promedio dentro de la misma localidad 

In [ ]:
viajes_mean_wait_sec['conteo'] = viajes_misma_locacion
viajes_mean_wait_sec_2 = viajes_mean_wait_sec.rename(columns={'mean':'tiempo_de_espera_promedio_sec'})
viajes_mean_wait_sec_2 = viajes_mean_wait_sec_2.sort_values('tiempo_de_espera_promedio_sec', ascending=False)
viajes_mean_wait_sec_2[viajes_mean_wait_sec_2['conteo'] > 4]

,tiempo_de_espera_promedio_sec,conteo
pickup_location,,
Benito Juárez,333.196581,351
Cuauhtémoc,277.897436,273
Azcapotzalco,277.759259,54
Coyoacán,252.660494,162
Miguel Hidalgo,246.290012,831
Iztacalco,242.217391,23
Venustiano Carranza,236.719178,146
Tlalnepantla de Baz,231.700000,10
La Magdalena Contreras,221.992308,260


Mayor cantidad de viajes dentro de la misma localidad

In [ ]:
viajes_mean_wait_sec_2.sort_values('conteo', ascending=False)

,tiempo_de_espera_promedio_sec,conteo
pickup_location,,
Iztapalapa,209.826132,972
Miguel Hidalgo,246.290012,831
Álvaro Obregón,213.222720,669
Benito Juárez,333.196581,351
Tlalpan,205.024316,329
Gustavo A. Madero,200.484099,283
Cuauhtémoc,277.897436,273
La Magdalena Contreras,221.992308,260
Xochimilco,182.370166,181


In [ ]:
### Mi función summary
def summary_taxis(nombre):
  minimo = all_taxis[nombre].min()
  Q1 = all_taxis[nombre].quantile(0.25)
  median = all_taxis[nombre].median()
  mean = all_taxis[nombre].mean()
  Q2 = all_taxis[nombre].quantile(0.5)
  Q3 = all_taxis[nombre].quantile(0.75)
  maximo = all_taxis[nombre].max()
  IQR = Q3 - Q1
  resumen = {'minimo': minimo, 'Q1': Q1, 'mediana':median,'Q2':Q2,'Q3':Q3,'max':maximo,'IQR':IQR, 'mean':mean}
  return resumen

In [ ]:
### Mi función summary
def summary_uber(nombre):
  minimo = all_uber[nombre].min()
  Q1 = all_uber[nombre].quantile(0.25)
  median = all_uber[nombre].median()
  mean = all_uber[nombre].mean()
  Q2 = all_uber[nombre].quantile(0.5)
  Q3 = all_uber[nombre].quantile(0.75)
  maximo = all_uber[nombre].max()
  IQR = Q3 - Q1
  resumen = {'minimo': minimo, 'Q1': Q1, 'mediana':median,'Q2':Q2,'Q3':Q3,'max':maximo,'IQR':IQR, 'mean':mean}
  return resumen

### Viajes solo por Taxis

In [ ]:
all_taxis = mun[mun['tipo_transporte'].str.startswith('R')+mun['tipo_transporte'].str.startswith('T')]
all_taxis.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "


,tipo_transporte,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,dist_meters,wait_sec,dist_km,speed_km,pickup_location,dropoff_location,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min,costo_viaje_sin_min,costo_viaje_sobre_min,viaje
id,,,,,,,,,,,,,,,,,,,,,
3,Taxi Libre,2016-09-18 10:11:50,2016-09-18 10:23:11,-99.289603,19.326263,-99.271874,19.328530,0.189167,2848,129,2.848,15.055507,Cuajimalpa de Morelos,Álvaro Obregón,8.74,1.07,1.07,8.74,23.996773,23.996773,mediano
4,Taxi Libre,2016-09-18 10:23:38,2016-09-18 10:30:53,-99.271161,19.328875,-99.279900,19.326256,0.121111,1409,106,1.409,11.633945,Álvaro Obregón,Álvaro Obregón,8.74,1.07,1.07,8.74,17.290964,17.290964,corto
5,Taxi Libre,2016-09-18 10:44:18,2016-09-18 10:51:40,-99.282761,19.326944,-99.291705,19.322754,0.122778,1567,85,1.567,12.762896,Álvaro Obregón,Cuajimalpa de Morelos,8.74,1.07,1.07,8.74,17.467871,17.467871,corto
6,Taxi Libre,2016-09-18 10:58:07,2016-09-18 10:59:46,-99.289712,19.322929,-99.289280,19.325978,0.027778,797,19,0.797,28.692000,Cuajimalpa de Morelos,Cuajimalpa de Morelos,8.74,1.07,1.07,8.74,12.602938,12.602938,corto
7,Taxi Libre,2016-09-18 11:00:00,2016-09-18 11:05:45,-99.285391,19.326247,-99.282549,19.328117,0.095833,676,169,0.676,7.053913,Álvaro Obregón,Álvaro Obregón,8.74,1.07,1.07,8.74,15.651724,15.651724,corto


In [ ]:
summary_taxis('speed_km')

{'IQR': 8.567702928610888,
 'Q1': 12.982099378881987,
 'Q2': 17.039049247156292,
 'Q3': 21.549802307492875,
 'max': 38.12604501607717,
 'mean': 17.61106477443831,
 'mediana': 17.039049247156292,
 'minimo': 0.7368340360291299}

In [ ]:
summary_taxis('wait_sec')

{'IQR': 277.25,
 'Q1': 86.0,
 'Q2': 190.0,
 'Q3': 363.25,
 'max': 901,
 'mean': 253.16160809371672,
 'mediana': 190.0,
 'minimo': 0}

### Viajes solo por Ubers

In [ ]:
all_uber = mun[mun['tipo_transporte'].str.startswith('U')]  
all_uber.head()

,tipo_transporte,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,dist_meters,wait_sec,dist_km,speed_km,pickup_location,dropoff_location,banderazo,tarifa_dist,tarifa_tiempo,tarifa_min,costo_viaje_sin_min,costo_viaje_sobre_min,viaje
id,,,,,,,,,,,,,,,,,,,,,
34,UberX,2016-09-19 02:04:43,2016-09-19 02:12:08,-99.162252,19.386298,-99.173536,19.389859,0.123889,2692,49,2.692,21.729148,Benito Juárez,Benito Juárez,7.00,3.57,1.80,35.0,18.080440,35.000000,mediano
139,UberBlack,2017-04-22 04:15:40,2017-04-22 04:30:05,-99.114205,19.292217,-99.069020,19.292230,0.240556,9061,40,9.061,37.666975,Tlalpan,Iztapalapa,30.00,9.46,3.50,77.0,118.050393,118.050393,largo
369,UberX,2017-04-30 02:32:13,2017-04-30 02:36:19,-99.212802,19.540852,-99.207274,19.543209,0.068333,2179,57,2.179,31.887805,Tlalnepantla de Baz,Tlalnepantla de Baz,7.00,3.57,1.80,35.0,16.489030,35.000000,mediano
609,UberXL,2017-03-31 10:46:26,2017-03-31 11:15:16,-99.098478,19.434810,-99.119207,19.382328,0.480556,12235,402,12.235,25.460116,Venustiano Carranza,Iztacalco,12.15,6.28,3.15,45.5,110.090800,110.090800,largo
707,UberSUV,2017-04-02 10:45:03,2017-04-02 10:49:34,-98.993944,19.267859,-98.993944,19.267859,0.075556,1468,119,1.468,19.429412,Tláhuac,Tláhuac,40.00,15.00,4.00,150.0,69.953333,150.000000,corto


In [ ]:
summary_uber('speed_km')

{'IQR': 12.306096256684496,
 'Q1': 11.072727272727272,
 'Q2': 18.39103869653768,
 'Q3': 23.37882352941177,
 'max': 37.86897810218978,
 'mean': 17.975000514458515,
 'mediana': 18.39103869653768,
 'minimo': 2.286999297259311}

In [ ]:
summary_uber('wait_sec')

{'IQR': 242.0,
 'Q1': 66.0,
 'Q2': 155.0,
 'Q3': 308.0,
 'max': 881,
 'mean': 218.48760330578511,
 'mediana': 155.0,
 'minimo': 0}

### Origen de los viajes

#### Taxis

In [ ]:
groupped_taxis_o = all_taxis.groupby('tipo_transporte')['pickup_location'].value_counts()
groupped_taxis_o

tipo_transporte  pickup_location        
Radio Taxi       Benito Juárez               459
                 Cuauhtémoc                  130
                 Miguel Hidalgo               64
                 Álvaro Obregón               37
                 Tlalpan                      21
                 Coyoacán                     18
                 La Magdalena Contreras       17
                 Azcapotzalco                 14
                 Iztapalapa                   11
                 Gustavo A. Madero            10
                 Tlalnepantla de Baz           8
                 Ecatepec de Morelos           7
                 Venustiano Carranza           6
                 Naucalpan de Juárez           5
                 Chalco                        2
                 Cuajimalpa de Morelos         2
                 Tecámac                       2
                 Cuautitlán                    1
                 Iztacalco                     1
                 Nezahualcóy

#### Ubers

In [ ]:
groupped_uber_o = all_uber.groupby('tipo_transporte')['pickup_location'].value_counts()
groupped_uber_o

tipo_transporte  pickup_location            
UberBlack        Álvaro Obregón                 2
                 Benito Juárez                  1
                 Miguel Hidalgo                 1
                 Nezahualcóyotl                 1
                 Querétaro                      1
                 Tlalpan                        1
                 Venustiano Carranza            1
UberSUV          Miguel Hidalgo                 3
                 Álvaro Obregón                 3
                 Benito Juárez                  2
                 Coyoacán                       2
                 Cuauhtémoc                     2
                 Iztapalapa                     2
                 Tláhuac                        2
                 Cuajimalpa de Morelos          1
                 Tulancingo de Bravo            1
UberX            Benito Juárez                  9
                 Coyoacán                       8
                 Gustavo A. Madero              7
     

### Destino de los viajes

#### Taxis

In [ ]:
groupped_taxis_d = all_uber.groupby('tipo_transporte')['dropoff_location'].value_counts()
groupped_taxis_d

tipo_transporte  dropoff_location           
UberBlack        Benito Juárez                  1
                 Gustavo A. Madero              1
                 Iztapalapa                     1
                 Miguel Hidalgo                 1
                 Nezahualcóyotl                 1
                 Querétaro                      1
                 Tlalpan                        1
                 Álvaro Obregón                 1
UberSUV          Miguel Hidalgo                 4
                 Iztapalapa                     2
                 Tláhuac                        2
                 Álvaro Obregón                 2
                 Azcapotzalco                   1
                 Benito Juárez                  1
                 Coyoacán                       1
                 Cuajimalpa de Morelos          1
                 Cuauhtémoc                     1
                 Tlalpan                        1
                 Tulancingo de Bravo            1
     

#### Ubers

In [ ]:
groupped_ubers_d = all_uber.groupby('tipo_transporte')['dropoff_location'].value_counts()
groupped_ubers_d

tipo_transporte  dropoff_location           
UberBlack        Benito Juárez                  1
                 Gustavo A. Madero              1
                 Iztapalapa                     1
                 Miguel Hidalgo                 1
                 Nezahualcóyotl                 1
                 Querétaro                      1
                 Tlalpan                        1
                 Álvaro Obregón                 1
UberSUV          Miguel Hidalgo                 4
                 Iztapalapa                     2
                 Tláhuac                        2
                 Álvaro Obregón                 2
                 Azcapotzalco                   1
                 Benito Juárez                  1
                 Coyoacán                       1
                 Cuajimalpa de Morelos          1
                 Cuauhtémoc                     1
                 Tlalpan                        1
                 Tulancingo de Bravo            1
     

### Análisis de tiempo de espera promedio en los viajes por día
La columna de `wait_sec` representa el tiempo en el viaje en el que el transporte estuvo totalmente detenido, esto puede ser usado como una medida del tráfico que se encontró durante el trayecto del viaje. En esta sección se realizará un análisis comparando el tiempo de espera de los viajes promedio por día para observar si cambia dependiendo cuál día de la semana es.

In [ ]:
data_municipios = data_costos.copy()

In [ ]:
days = {0:'Lunes',1:'Martes',2:'Miercoles',3:'Jueves',4:'Viernes',5:'Sabado',6:'Domingo'}
data_municipios["day_of_trip"] = data_municipios["pickup_datetime"].dt.dayofweek
data_municipios["day_of_trip"] = data_municipios["day_of_trip"].apply(lambda x: days[x])

In [ ]:
data_municipios["day_of_trip"].sample(5)

id
11677       Sabado
1370     Miercoles
6802     Miercoles
789         Martes
12569       Sabado
Name: day_of_trip, dtype: object

In [ ]:
table = data_municipios.groupby(["day_of_trip"])["wait_sec"].mean()
table = table[[0,2,3,4,1,6,5]]
pd.DataFrame(table)

,wait_sec
day_of_trip,
Domingo,239.119048
Lunes,251.316308
Martes,263.925866
Miercoles,239.647149
Jueves,261.868038
Viernes,264.780423
Sabado,241.573290


Se puede apreciar que el viernes es el día con más tráfico. Sin embargo, para confirmar esto, es necesario un análisis más profundo de los datos.

### Análisis de cantidad de viajes por cada día de la semana.

In [ ]:
table = data_municipios.groupby(["day_of_trip"])["wait_sec"].count()
table = table[[0,2,3,4,1,6,5]]
pd.DataFrame(table)

,wait_sec
day_of_trip,
Domingo,756
Lunes,1116
Martes,1241
Miercoles,1298
Jueves,1167
Viernes,1134
Sabado,921


Se aprecia que el Miercoles es el día que más se tiene demanda de viajes en transporte privado.

### Comparación de tiempo detenido en el tráfico por viaje en Taxis y Ubers.
Se comparará el tiempo de espera promedio por viaje de los transporte tipo Uber y Taxis.

In [ ]:
data_municipios["tipo_transporte"].unique()

array(['Taxi Libre', 'Radio Taxi', 'Taxi de Sitio', 'UberX', 'UberBlack',
       'UberXL', 'UberSUV'], dtype=object)

In [ ]:
taxis = ['Taxi Libre', 'Radio Taxi', 'Taxi de Sitio']

Se inicializa una lista que contiene los diferentes tipos de transporte taxis para clasificarlos a travez del método `apply` solamente como *Taxi* y *Uber*

In [ ]:
data_transporte = data_municipios.copy()

In [ ]:
data_transporte["tipo_transporte"] = data_transporte["tipo_transporte"].apply(lambda x: "Taxi" if x in taxis else "Uber")

In [ ]:
data_transporte["tipo_transporte"].unique()

array(['Taxi', 'Uber'], dtype=object)

In [ ]:
table = data_transporte.groupby(["tipo_transporte"])["wait_sec"].mean()
pd.DataFrame(table)

,wait_sec
tipo_transporte,
Taxi,253.161608
Uber,218.487603


Se puede ver que los transportes tipo *Uber* pasan menos tiempo detenidos en el tráfico que los *Taxis*. Esto posiblemente se debe a que los *Uber* usualmente se utilizan aplicaciones para evitar el tráfico.

### Velocidad Promedio por cada tipo de transporte

In [ ]:
table = data_transporte.groupby(["tipo_transporte"])["speed_km"].mean()
pd.DataFrame(table)

,speed_km
tipo_transporte,
Taxi,17.611065
Uber,17.975001


A pesar de que aparentemente los transportes tipo *Uber* son más veloces, es muy pequeña la diferencia como para asumir que es verdad. Se necesita un estudio estadístico más profundo para confirmar esto.

# 7. Conclusión
A partir del análisis exploratorio que se realizó, se observo que los transportes Uber son más rápidos que los taxis y pasan menos tiempo detenidos en el tráfico, sin embargo también son mas costosos. Los datos presentados en este trabajo son preliminares, por lo tanto se necesita un análisis mas profundo. 

Si en algún análisis posterior se necesitara usar métodos de inteligencia artificial o aprendizaje profundo podrán usar el dataset resultante del proceso de limpieza que se realizó en este notebook, el cual está almacenado en un repositorio de [GitHub](https://github.com/CristopherCano/Proyecto_Python_Procesamiento_de_datos/blob/main/Data/cdmx_transporte.csv).